# old vs new

In [14]:
import redis
import numpy
import math
import transvis.io as t
import matplotlib.pyplot as plt
import scipy
import scipy.stats
%matplotlib inline
import plotly.offline as off
off.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [16]:
def name_to_plot(name):
    t.initDB()
    MAL = [m for m, _, _, _ in t.alleleData()]
    def avg(data):
        return sum(data)/len(data)

    def lez(a):
        if a < 0:
            return -1
        else:
            return 1
        import scipy.stats

    slope_pvalue_intensity_position = []
    for probeset_key in t.getGenecodeGenenameToProbeset(name):
        probesets = t.probesetData(probeset_key)
        slopes = []
        pvalues = []
        for probeset in probesets:
            probeset = [math.log(i) for i in probeset]
            slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(MAL, probeset)
            left = t.probesetAnnotationMetadata(probeset_key)[3]
            slope_pvalue_intensity_position.append([slope, p_value, avg(probeset), left, probeset_key])

    zipped_properties = list(zip(*slope_pvalue_intensity_position))
    slope, intensity, pvalues, position, probeset_belongs = zipped_properties[0], zipped_properties[2], zipped_properties[1], zipped_properties[3], zipped_properties[4]    
    signed_pvalues = [-math.log(pvalue, 10) * lez(s) for pvalue, s in zip(pvalues, slope)]
    
    data = [dict(
      type = 'scatter',
      x = position,
      y = signed_pvalues,
      mode = 'markers',
      transforms = [dict(
        type = 'groupby',
        groups = probeset_belongs
      )]
    ),
    go.Scatter(
      x = position,
      y = [-math.log(0.05, 10)] * len(position),
      mode = 'lines',
      name = 'down @ 0.05'
    ),
    go.Scatter(
      x = position,
      y = [math.log(0.05, 10)] * len(position),
      mode = 'lines',
      name = 'up @ 0.05'
    ),
    go.Scatter(
      x = position,
      y = [-math.log(0.05/len(position), 10)] * len(position),
      mode = 'lines',
      name = 'down @ Bonferroni 0.05'
    ),
    go.Scatter(
      x = position,
      y = [math.log(0.05/len(position), 10)] * len(position),
      mode = 'lines',
      name = 'up @ Bonferroni 0.05'
    )]

    #off.iplot({'data': data}, validate=False)
    off.iplot({'data': data, 'layout': dict(title = name)}, validate=False)

In [17]:
import os
import re
import numpy
import math
import matplotlib.pyplot as plt
import scipy
import scipy.stats
%matplotlib inline
import plotly.offline as off
off.init_notebook_mode(connected=True)
import plotly.graph_objs as go

directory = "experiment_muscle"

gene = "TNNI1"

# copied from quantile_normalise
def load_metadata():
    metadata = {}
    metadata_order = []
    with open("metadata.txt") as f:
        for i, line in enumerate(f):
            line = line.strip().split()
            if i == 0:
                names = line[1:]
            else:
                values = line[1:]
                patient_id = line[0]
                metadata_order.append(patient_id)
                metadata[patient_id] = {k: v for k, v in zip(names, values)}
    return metadata_order, metadata

metadata_order, metadata = load_metadata()
blood_IDs = [i for i in metadata_order]
blood_CELs = [metadata[i]["blood_cel"] for i in blood_IDs]
blood_MA = [metadata[i]["modal_allele"] for i in blood_IDs]
blood_MA = [int(i) for i in blood_MA]
blood_PA = [metadata[i]["progenitor_allele"] for i in blood_IDs]
blood_MIRS = [metadata[i]["MIRS"] for i in blood_IDs]

muscle_IDs = [i for i in metadata_order if metadata[i]["muscle_cel"] != "refused_biopsy"]
muscle_CELs = [metadata[i]["muscle_cel"] for i in muscle_IDs]
muscle_MA = [metadata[i]["modal_allele"] for i in muscle_IDs]
muscle_MA = [int(i) for i in muscle_MA]

muscle_PA = [metadata[i]["progenitor_allele"] for i in muscle_IDs]
muscle_MIRS = [metadata[i]["MIRS"] for i in muscle_IDs]

probe_data = []
probe_IDs = []

with open(os.path.join(directory, gene)) as f:
    for i, line in enumerate(f):
        line = line.rstrip().split()
        if i == 0:
            prefix = "patient_"
            our_muscle_IDs = [elem[len(prefix):] for elem in line if re.match(prefix, elem)]
            #print(len(blood_IDs), blood_IDs)
            #print(len(blood_IDs), our_blood_IDs)
            assert muscle_IDs == our_muscle_IDs
            headers = {header: i for i, header in enumerate(line)}
            patient_data = {header[len(prefix):]: i for i, header in enumerate(line) if re.match(prefix, header)}
            def write_signature(line):
                signature = {}
                for elem in ["gene_name", "probeset_id", "seq5to3plus", "chrom", "strand", "genocode_left", "genecode_right"]:
                    signature[elem] = line[headers[elem]]
                return signature
        else:
            probe_ID = write_signature(line)
            rv = []
            for patient_id in muscle_IDs:
                rv.append(float(line[patient_data[patient_id]]))
            probe_data.append(rv)
            probe_IDs.append(probe_ID)

probe_data

probe_IDs


MAL = muscle_MA
def plot(probe_data):
    def avg(data):
        return sum(data)/len(data)

    def lez(a):
        if a < 0:
            return -1
        else:
            return 1
        import scipy.stats

    slope_pvalue_intensity_position = []
    for i, probeset in enumerate(probe_data):
        probeset = [math.log(i) for i in probeset]
        left = probe_IDs[i]["genocode_left"]
        probeset_key = probe_IDs[i]["probeset_id"]
        slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(MAL, probeset)
        slope_pvalue_intensity_position.append([slope, p_value, avg(probeset), left, probeset_key])
        

    zipped_properties = list(zip(*slope_pvalue_intensity_position))
    slope, intensity, pvalues, position, probeset_belongs = zipped_properties[0], zipped_properties[2], zipped_properties[1], zipped_properties[3], zipped_properties[4]
    signed_pvalues = [-math.log(pvalue, 10) * lez(s) for pvalue, s in zip(pvalues, slope)]
    
    data = [dict(
      type = 'scatter',
      x = position,
      y = signed_pvalues,
      mode = 'markers',
      transforms = [dict(
        type = 'groupby',
        groups = probeset_belongs
      )]
    ),
    go.Scatter(
      x = position,
      y = [-math.log(0.05, 10)] * len(position),
      mode = 'lines',
      name = 'down @ 0.05'
    ),
    go.Scatter(
      x = position,
      y = [math.log(0.05, 10)] * len(position),
      mode = 'lines',
      name = 'up @ 0.05'
    ),
    go.Scatter(
      x = position,
      y = [-math.log(0.05/len(position), 10)] * len(position),
      mode = 'lines',
      name = 'down @ Bonferroni 0.05'
    ),
    go.Scatter(
      x = position,
      y = [math.log(0.05/len(position), 10)] * len(position),
      mode = 'lines',
      name = 'up @ Bonferroni 0.05'
    )]

    #off.iplot({'data': data}, validate=False)
    off.iplot({'data': data, 'layout': dict(title = gene)}, validate=False)

In [18]:
gene = "TNNI1"

In [19]:
name_to_plot(gene)

In [22]:
plot(probe_data)